  
# Trabajo Práctico 1

## Simulación de Eventos Discretos

#### Departamento de Computación - Facultad de Ciencias Exactas y Naturales - Universidad de Buenos Aires

20 de Septiembre de 2021

**Grupo 04:** Confalonieri, Gisela - Lucero, Emiliano
    

----

# Modelo conceptual

En el presente trabajo modelaremos un **sistema de evaluación de un centro de atención telefónica de una entidad bancaria**. 

Una **simplificación** de este tipo de sistemas podría ser la siguiente:

* Una persona llama a la central telefónica de la entidad bancaria.
* Una contestadora automática solicita datos a la persona para derivarla al área correspondiente a su solicitud.
* El llamado es colocado en espera para ser atendido por un agente del área correspondiente.
* El agente atiende la llamada y resuelve la solicitud.
* Durante el recorrido de la llamada a través del sistema, se recopilan datos que serán utilizados en la generación de métricas de calidad de servicio.

El objetivo de nuestro modelo es **comparar posibles configuraciones en cuanto a la disposición de los agentes en cada una de las áreas de atención, para evaluar los cambios de performance (tiempo de demora en resolver una solicitud) según las métricas obtenidas**. Las posteriores simulaciones permitirían **analizar distintos escenarios en los que se observen mejoras en el servicio.**

## Estructura del modelo

Nuestro modelo comprenderá:

* Un **generador de las llamadas** que ingresan al centro de atención
* Una **mesa de entrada** que redirigirá las llamadas al área correspondiente
* Dos **áreas de atención específicas**, una para clientes y otra para no clientes
* Un **medidor de métricas de calidad**

Por lo tanto, un primer bosquejo de la estructura de nuestro modelo comprende los siguientes componentes:

*Esquema general del modelo*

![esquema general](https://raw.githubusercontent.com/giselabelen/SED2021/master/imgs/SED2021-TP1-General.png)

### Generador de llamadas

* Representa las llamadas que recibe el centro de atención.
* Su único input corresponde a un evento que indica cuándo dejar de generar llamados.
* Periódicamente genera llamadas según alguna distribución.
* Cada llamada generada corresponde a un cliente o a una persona que aún no es cliente.

*Esquema del generador de llamadas*
  
![esquema general](https://raw.githubusercontent.com/giselabelen/SED2021/master/imgs/SED2021-TP1-Generador.png)

#### **Especificación formal**

$ generadorDeLlamadas = <X,Y,S,\delta_{int},\delta_{ext},\lambda,ta> $

$X = \{parar\}$

$Y = \{llamada\}$

$S = \{G\}$

$\delta_{int}(G) = G$

$\delta_{ext}(G,e,parar) =$ ```passivate```

$\lambda(s) =$ 
```c++
id <- número entero identificatorio //único para cada llamada generada en una simulación
es_cliente <- booleano aleatorio //true si es llamada de cliente, false para no cliente
enviar como output la tupla [id,es_cliente]
```

$ta(G) = $ frecuencia_generacion

### Mesa de entrada

* Recibe las llamadas como input, y las redirige al área correspondiente.
  * Las llamadas ingresan a una cola de espera.
  * Un subcomponente va desencolando las llamadas y enviándolas al área correspondiente.

*Esquema de la mesa de entrada*
  
![esquema general](https://raw.githubusercontent.com/giselabelen/SED2021/master/imgs/SED2021-TP1-mesa_de_entrada.png)

#### Especificación
##### **Distribuidor llamadas**

$ Distribuidor Llamadas = <X,Y,S,\delta_{int},\delta_{ext},\lambda,ta> $

$X = \{entrante\}$

$Y = \{descolar, clientes, noClientes\}$

$S = \{requesting, waiting, sending\}$

$\delta_{int}(requesting) = waiting$

$\delta_{int}(sending) = requesting$

$\delta_{ext}(waiting, e, entrante) = sending$

$\delta_{ext}(s, e, x) =$ 

```
 if waiting {
      Preparo para enviarle una llamada a un área
 }
 else {
     // No puedo recibir si no pedi antes
     error
 }

```

$\lambda(s) =$ 
```
case s:
     requesting:
         Send 1 to descolar
     Waiting:
         // No hacer nada, sigo esperando a la cola
     sending:
         // Recibi una llamada de la cola
         send llamada to área correspondiente
     otherwise:
         error
         
```

$ta(waiting) =$ Infinity

$ta(requesting) =$ 0

$ta(sending) =$ delay

### Áreas de atención específica

* Cada una de las áreas recibe como input las llamadas que llegan desde la mesa de entrada.
* Estas llamadas ingresan a una cola de espera.
* Un distrbuidor va desencolando las llamadas y asignándolas a los agentes que se encuentren disponibles.
* Cada agente atiende una llamada y resuelve la solicitud. 
* Cada área comunica, a través de sus outputs, el momento en que cada llamada es atendida por un agente y el momento en que cada agente resuelve la solicitud de una llamada y la misma finaliza.

*Esquema de las áreas de atención específica*
  
![esquema general](https://raw.githubusercontent.com/giselabelen/SED2021/master/imgs/SED2021-TP1-areas_especificas.png)

#### Especificación
##### **Distribuidor agentes**

$ Distribuidor Agentes = <X,Y,S,\delta_{int},\delta_{ext},\lambda,ta> $

$X = \{peticion, entrante\}$

$Y = \{descolar, agente1, agente2\}$

$S = \{idle, requesting, waiting, sending\}$

$\delta_{int}(idle) = idle$

$\delta_{int}(idle) =$
```
if no hay pedidos encolador {
     passivate
 } else {
     paso al estado requesting
     ta = 0\}$
 }
```

$\delta_{int}(requesting) = waiting$

$\delta_{int}(sending) = idle$

$\delta_{ext}(idle, e, peticion) = requesting$

$\delta_{ext}(waiting, e, entrante) = sending$

$\delta_{ext}(s, e, x) =$ 

```
case port:
     peticion:
         Encolo pedido de llamada para el agente i
         if idle {
            // Empiezo a realizar pedidos
         } 
         else {
            // Ya estoy realizando pedidos, no hago nada
            Encolo el pedido
            Mantengo estado y ta
         }
     entrante:
         if waiting {
              Preparo para enviarle una llamada a un agente i
              Desecolo pedido del agente i ya que llego la llamada
         }
         else {
             // No puedo recibir si no pedi antes
             error
         }

```

$\lambda(s) =$ 
```
case s:
     idle:
         // No hacer nada
         
     requesting:
         send 1 to descolar
     waiting:
         // No hacer nada, sigo esperando a la cola
     sending:
         // Recibi una llamada de la cola
         send llamada to agente i
     otherwise:
         error
         
```

$ta(passive) =$ Infinity

$ta(active) =$ frecuenciaDistribuidor

##### **Agente**

$ Agente = <X,Y,S,\delta_{int},\delta_{ext},\lambda,ta> $

$X = \{entrante\}$

$Y = \{pedido, finalizada\}$

$S = \{atendiendo, pidiendo\}$

$\delta_{int}(atendiendo) = pidiendo$

$\delta_{ext}(pidiendo, e, entrante) = atendiendo$

$\delta_{ext}(s, e, x) =$ 

```
// No puede pasar que me llegue una llamada mientras atiendo otra
if atendiendo {
   error
}

// Estaba libre, atiendo
if pidiendo {
   frecuenciaAgente = valor aleatorio dado por la distribución elegida
}
```

$\lambda(active) =$ 
```
// Estaba atendiendo y ya transcurrió ta, por lo que terminé de atender la llamada
// La libero y aviso que estoy libre, pidiendo que me pasen otra llamada
send idAgente to pedido
if termine llamada {
     send llamada to finalizada
}
```

$ta(pidiendo) =$ Infinity

$ta(atendiendo) =$ frecuenciaAgente

### Medidor de métricas

* Comprende diversos subcomponentes que evaluarán distintas métricas de calidad cada uno, a partir de los datos recibidos a lo largo del recorrido de la llamada por el centro de atención telefónica, a saber:
  * Momento en que arriba una llamada al servicio
  * Momento en que la llamada es atendida por un agente
  * Momento en que la solicitud es resuelta por un agente
* Nuestro primer bosquejo comprende las siguientes métricas:
  * Tiempo promedio que una llamada está aguardando ser atendida (es decir, desde que llega hasta que es atendida por un agente)
  * Tiempo máximo que una llamada estuvo aguardando ser atendida (es decir, desde que llega hasta que es atendida por un agente)
  * Tiempo promedio que una llamada consume dentro del centro de atención (es decir, desde que llega hasta que su solicitud se resuelve)
  * Tiempo máximo que una llamada consumió dentro del centro de atención (es decir, desde que llega hasta que su solicitud se resuelve)

*Esquema de los medidores de métricas*
  
![esquema general](https://raw.githubusercontent.com/giselabelen/SED2021/master/imgs/SED2021-TP1-Medidor.png)

#### Especificación

$ Medidor = <X,Y,S,\delta_{int},\delta_{ext},\lambda,ta> $

$X = \{entrante, atendida, finalizada\}$

$Y = \{mediciones\}$

$S = \{midiendo\}$

$\delta_{int}(midiendo) = midiendo$

$\delta_{ext}(midiendo, e, entrante) = midiendo$

$\delta_{ext}(midiendo, e, atendida) = midiendo$

$\delta_{ext}(midiendo, e, finalizada) = midiendo$

$\delta_{ext}(s, e, x) =$ 

```
case port:
     entrante:
         Guardamos el timestamp de entrada segun el id de la llamada
         Sumamos 1 al contador de llamadas totales
     atendida:
         Guardamos el timestamp de atencion segun el id de la llamada
         Sumamos 1 al contador de llamadas atendidas
         Calculamos promedio y máximo de tiempos de llamadas atendidas (PAtendidas y MaxAtendidas)
     finalizada:
         Guardamos el timestamp de finalizacion segun el id de la llamada
         Sumamos 1 al contador de llamadas finalizadas
         Calculamos promedio y máximo de tiempos de llamadas finalizadas (PFinalizadas y MaxFinalizadas)
     otherwise:
         Error
```

$\lambda(midiendo) =$ 
```
m = <#Total, #atendidas, #finalizadas, PAtendidas, MaxAtendidas, PFinalizadas, MaxFinalizadas>
Send m to mediciones

```

$ta(midiendo) =$ frecuencia_reporte